<a href="https://colab.research.google.com/github/jitendra-manwani/GenAI-architect/blob/main/Jitendra_Assignmengt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Assignment 4
# Simplified Agentic RAG System with LangGraph


# =====================================
# STEP 1: Install Required Packages
# =====================================

!pip install langgraph openai chromadb sentence-transformers pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.5 MB/s eta 0:0

In [2]:
# =====================================
# STEP 2: Import Libraries and Setup
# =====================================

import json
import os
from typing import List, Dict, Any
import openai
from openai import AzureOpenAI
import chromadb
from sentence_transformers import SentenceTransformer
from langgraph.graph import Graph
import uuid

In [6]:
# Set up Azure OpenAI configuration
# Replace these with your actual Azure OpenAI credentials
AZURE_OPENAI_ENDPOINT = "https://eastus.api.cognitive.microsoft.com/"
AZURE_OPENAI_KEY = "63a0587ef7b24ca9b5d7b179c13fbaab"
AZURE_DEPLOYMENT_NAME = "telcogpt"

# Create Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version="2024-12-01-preview"
)

In [16]:
# Initialize embedding model (using sentence-transformers for simplicity)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Chroma DB
chroma_client = chromadb.Client()

print("Setup complete!")

Setup complete!


In [7]:
# =====================================
# STEP 3: Create Sample Dataset
# =====================================

# Since we don't have the actual JSON file, let's create a sample dataset
sample_kb_data = [
    {
        "doc_id": "KB001",
        "question": "What are best practices for debugging?",
        "answer_snippet": "When debugging, start with a minimal reproducible test case, use systematic logging at key points, apply divide-and-conquer approach to isolate the issue, and use debugger tools effectively.",
        "source": "debugging_guide.md",
        "last_updated": "2024-01-10"
    },
    {
        "doc_id": "KB002",
        "question": "What are performance tuning tips?",
        "answer_snippet": "For performance tuning, profile your application first to identify bottlenecks, optimize database queries, implement caching strategies, and monitor resource usage continuously.",
        "source": "performance_guide.md",
        "last_updated": "2024-01-15"
    },
    {
        "doc_id": "KB003",
        "question": "What are best practices for caching?",
        "answer_snippet": "Implement cache-aside pattern, set appropriate TTL values, use cache hierarchies for different data types, and implement cache warming strategies for critical data.",
        "source": "caching_guide.md",
        "last_updated": "2024-01-20"
    },
    {
        "doc_id": "KB004",
        "question": "How should I set up CI/CD pipelines?",
        "answer_snippet": "Set up automated testing at multiple levels, use infrastructure as code, implement gradual rollouts, and maintain separate environments for development, staging, and production.",
        "source": "cicd_guide.md",
        "last_updated": "2024-01-25"
    },
    {
        "doc_id": "KB005",
        "question": "How do I version my APIs?",
        "answer_snippet": "Use semantic versioning (major.minor.patch), maintain backward compatibility when possible, provide clear migration guides, and deprecate old versions gradually.",
        "source": "api_guide.md",
        "last_updated": "2024-02-01"
    },
    {
        "doc_id": "KB006",
        "question": "What should I consider for error handling?",
        "answer_snippet": "Implement structured exception handling, log errors with sufficient context, provide meaningful error messages to users, and implement retry mechanisms for transient failures.",
        "source": "error_handling_guide.md",
        "last_updated": "2024-02-05"
    },
    {
        "doc_id": "KB007",
        "question": "What are CI/CD pipeline best practices?",
        "answer_snippet": "Use automated testing, implement blue-green deployments, maintain deployment scripts in version control, and monitor deployment metrics.",
        "source": "pipeline_guide.md",
        "last_updated": "2024-02-10"
    },
    {
        "doc_id": "KB008",
        "question": "How to implement cache invalidation?",
        "answer_snippet": "Use time-based expiration, implement cache tags for grouped invalidation, use event-driven invalidation, and consider cache coherence in distributed systems.",
        "source": "cache_invalidation_guide.md",
        "last_updated": "2024-02-12"
    },
    {
        "doc_id": "KB009",
        "question": "What are profiling tools for performance?",
        "answer_snippet": "Use application performance monitoring tools, implement custom metrics, use database query analyzers, and monitor system resource usage with tools like htop or Performance Monitor.",
        "source": "profiling_guide.md",
        "last_updated": "2024-02-15"
    },
    {
        "doc_id": "KB010",
        "question": "What is semantic versioning?",
        "answer_snippet": "Semantic versioning uses MAJOR.MINOR.PATCH format where MAJOR version changes break compatibility, MINOR version adds functionality, and PATCH version fixes bugs.",
        "source": "versioning_guide.md",
        "last_updated": "2024-02-18"
    }
]

print(f"Created sample dataset with {len(sample_kb_data)} entries")

Created sample dataset with 10 entries


In [8]:
# =====================================
# CELL 4: Index Knowledge Base
# =====================================

# Create or get collection
try:
    collection = chroma_client.create_collection(name="kb_collection")
except:
    chroma_client.delete_collection(name="kb_collection")
    collection = chroma_client.create_collection(name="kb_collection")

# Process and index each document
documents = []
metadatas = []
ids = []
embeddings = []

for item in sample_kb_data:
    # Generate embedding for the answer snippet
    embedding = embedding_model.encode(item["answer_snippet"]).tolist()

    documents.append(item["answer_snippet"])
    metadatas.append({
        "doc_id": item["doc_id"],
        "source": item["source"],
        "last_updated": item["last_updated"],
        "question": item["question"]
    })
    ids.append(item["doc_id"])
    embeddings.append(embedding)

# Add to collection
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print(f"Indexed {len(documents)} documents successfully!")

Indexed 10 documents successfully!


In [13]:
# =====================================
# STEP 5: Define RAG Components
# =====================================

def retrieve_kb(user_question: str, top_k: int = 5) -> List[Dict]:
    """Retrieve top-k relevant documents from knowledge base"""
    query_embedding = embedding_model.encode(user_question).tolist()

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    retrieved_docs = []
    for i in range(len(results['ids'][0])):
        retrieved_docs.append({
            "doc_id": results['metadatas'][0][i]['doc_id'],
            "answer_snippet": results['documents'][0][i],
            "source": results['metadatas'][0][i]['source'],
            "distance": results['distances'][0][i]
        })

    return retrieved_docs

def generate_answer(user_question: str, kb_hits: List[Dict]) -> str:
    """Generate initial answer using retrieved context"""

    # Format retrieved snippets
    snippets_text = ""
    for hit in kb_hits:
        snippets_text += f"[{hit['doc_id']}] {hit['answer_snippet']}\n\n"

    prompt = f"""You are a software best-practices assistant.

User Question:
{user_question}

Retrieved Snippets:
{snippets_text}

Task:
Based on these snippets, write a concise answer to the user's question.
Cite each snippet you use by its doc_id in square brackets (e.g., [KB004]).
Return only the answer text."""

    response = client.chat.completions.create(
        model=AZURE_DEPLOYMENT_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content

def critique_answer(user_question: str, initial_answer: str, kb_hits: List[Dict]) -> str:
    """Critique the initial answer for completeness"""

    # Format KB snippets
    snippets_text = ""
    for hit in kb_hits:
        snippets_text += f"[{hit['doc_id']}] {hit['answer_snippet']}\n\n"

    prompt = f"""You are a critical QA assistant. The user asked: {user_question}

Initial Answer:
{initial_answer}

KB Snippets:
{snippets_text}

Task:
Determine if the initial answer fully addresses the question using only these snippets.
- If it does, respond exactly: COMPLETE
- If it misses any point or cites missing info, respond: REFINE: <short list of missing topic keywords>

Return exactly one line."""

    response = client.chat.completions.create(
        model=AZURE_DEPLOYMENT_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()

def refine_answer(user_question: str, initial_answer: str, critique_result: str, kb_hits: List[Dict]) -> str:
    """Refine answer based on critique"""

    # Extract missing keywords from critique
    missing_keywords = critique_result.replace("REFINE:", "").strip()

    # Build new query and retrieve additional snippet
    new_query = f"{user_question} {missing_keywords}"
    additional_snippet = retrieve_kb(new_query, top_k=1)[0]

    prompt = f"""You are a software best-practices assistant refining your answer.

User Question: {user_question}

Initial Answer:
{initial_answer}

Critique: {critique_result}

Additional Snippet:
[{additional_snippet['doc_id']}] {additional_snippet['answer_snippet']}

Task:
Incorporate this snippet into the answer, covering the missing points.
Cite any snippet you use by doc_id in square brackets.
Return only the final refined answer."""

    response = client.chat.completions.create(
        model=AZURE_DEPLOYMENT_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content

print("RAG components defined successfully!")

RAG components defined successfully!


In [14]:
# =====================================
# CELL 6: Test the Pipeline with Sample Queries
# =====================================

# Test queries from the assignment
test_queries = [
    "What are best practices for caching?",
    "How should I set up CI/CD pipelines?",
    "What are performance tuning tips?",
    "How do I version my APIs?",
    "What should I consider for error handling?"
]

# Process each query
for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {query}")
    print('='*60)

    # Step 1: Retrieve relevant documents
    print("\n1. RETRIEVING RELEVANT DOCUMENTS...")
    kb_hits = retrieve_kb(query, top_k=5)

    for j, hit in enumerate(kb_hits):
        print(f"   [{hit['doc_id']}] Score: {hit['distance']:.3f}")
        print(f"   {hit['answer_snippet'][:100]}...")
        print()

    # Step 2: Generate initial answer
    print("2. GENERATING INITIAL ANSWER...")
    initial_answer = generate_answer(query, kb_hits)
    print(f"Initial Answer: {initial_answer}")

    # Step 3: Critique the answer
    print("\n3. CRITIQUING ANSWER...")
    critique_result = critique_answer(query, initial_answer, kb_hits)
    print(f"Critique Result: {critique_result}")

    # Step 4: Refine if needed
    final_answer = initial_answer

    if critique_result.startswith("REFINE"):
        print("\n4. REFINING ANSWER...")
        final_answer = refine_answer(query, initial_answer, critique_result, kb_hits)
        print(f"Refined Answer: {final_answer}")
    else:
        print("\n4. NO REFINEMENT NEEDED")

    # Final JSON response
    final_response = {"answer": final_answer}
    print(f"\n5. FINAL JSON RESPONSE:")
    print(json.dumps(final_response, indent=2))



TEST QUERY 1: What are best practices for caching?

1. RETRIEVING RELEVANT DOCUMENTS...
   [KB003] Score: 0.701
   Implement cache-aside pattern, set appropriate TTL values, use cache hierarchies for different data ...

   [KB008] Score: 0.738
   Use time-based expiration, implement cache tags for grouped invalidation, use event-driven invalidat...

   [KB002] Score: 1.071
   For performance tuning, profile your application first to identify bottlenecks, optimize database qu...

   [KB009] Score: 1.403
   Use application performance monitoring tools, implement custom metrics, use database query analyzers...

   [KB005] Score: 1.406
   Use semantic versioning (major.minor.patch), maintain backward compatibility when possible, provide ...

2. GENERATING INITIAL ANSWER...
Initial Answer: Best practices for caching include implementing the cache-aside pattern, setting appropriate time-to-live (TTL) values, and using cache hierarchies for different data types. Additionally, consider cache 